<a href="https://colab.research.google.com/github/latterfate/hello-world/blob/master/hollister_project_version_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
import re
import nltk
from nltk.stem.porter import *
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
import gensim
from gensim.models import Word2Vec
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import precision_recall_fscore_support
from statistics import mean
from numpy import array
from numpy import asarray
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import zeros
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow import keras

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Import data

There are four features we can extract from the table below: title, lens, summary and linkref. In the first trial we will only use title and summary for text classification.

In [4]:
df = pd.read_excel('drive/MyDrive/Hollister Confidential-Futurism 2021 - DeDuplication Tool v01c (1).xlsx', sheet_name='Clues')
df.head()

,id,title,Lens,summaryplaintext,linkRef
0,2772,CC project for futurism started,Political,Futurism gaining interest within the organizat...,Reference:&nbsp;check FEI records
1,2773,DELETE ME Great eagerness tp participate from...,Societal,opt-in participation is not meant to stress pe...,NaN
2,2774,"this is what I heard the customer say, ""La di ...",Environmental,I think they meant that ehy wanted something t...,<div>hyperlink 1</div><div>'KEEP THIS AREA BRI...
3,2789,"Reducitarian Kateman: ""Technological innovatio...",Economical,There is growing awareness that sustainability...,"<p><a href=""https://www.fastcompany.com/905995..."
4,2790,Medical Futurist Meskó broadly covers wearable...,Social,Message is that healthcare is still trying to ...,<div>1. https://open.spotify.com/episode/4OtPk...


# Preprocess title + summary text

In [5]:
# Concat title and summary plain text
df['full_text'] = df['title'] + ' ' + df['summaryplaintext']

### remove non algebra characters

In [6]:
# Convert to lowercase
full_text_ls = [df['full_text'][i].lower() for i in range(len(df['full_text']))]

# Remove numbers / punctuation / special characters
full_text_ls = [re.sub('[^A-Za-z]+', ' ', i) for i in full_text_ls]

# Strip white space
full_text_ls = [i.strip() for i in full_text_ls]

### conduct stemming

In [7]:
stemmer = PorterStemmer()
stemmed_text = []
for string in full_text_ls:
  new_string = ' '.join(stemmer.stem(token) for token in nltk.word_tokenize(string))
  stemmed_text.append(new_string)

print(stemmed_text)


['cc project for futur start futur gain interest within the organ delet thi clue after test', 'delet me great eager tp particip from respond opt in particip is not meant to stress peopl out but rather integr into their work the selef select mental lead to inprov qualiti of output', 'thi is what i heard the custom say la di da di i think they meant that ehi want someth thinnner not necessarili shorter', 'reducitarian kateman technolog innov ha a role to play in advanc moral progress but so do ethic argument there is grow awar that sustain and climat chang will not be address without the buy in of corpor strategi', 'medic futurist mesk broadli cover wearabl microbiom and ai in recent podcast messag is that healthcar is still tri to figur out what to do with the insight provid by the tidal wave of data that is readili avail to patient consum believ that a cultur transform in healthcar is need befor wearabl data becom usabl', 'darpa award m in contract to alter microbiom to repel mosquito 

In [8]:
df['cleaned_text'] = stemmed_text

### create dependent variable

In [9]:
# load hunch data and count clue frequency
hunch = pd.read_excel('drive/MyDrive/Hollister Confidential-Futurism 2021 - DeDuplication Tool v01c (1).xlsx', sheet_name='Hunches')
hunch[['Clue1', 'Clue2','Clue3', 'Clue4','Clue5']] = hunch[['Clue1', 'Clue2','Clue3', 'Clue4','Clue5']].fillna(0).astype(int)
all_clue = pd.concat([hunch['Clue1'],hunch['Clue2'], hunch['Clue3'], hunch['Clue4'], hunch['Clue5']], ignore_index=True)
clue_frequency = all_clue.value_counts()

In [10]:
del clue_frequency[0]
clue_frequency = pd.DataFrame(clue_frequency, columns= ['frequency'])
clue_frequency['id'] = clue_frequency.index

In [11]:
# fill un-used clue with 0
df = df.merge(clue_frequency, how = 'left', on = 'id')
df['frequency'] = df['frequency'].fillna(0).astype(int)

In [12]:
pd.options.display.max_colwidth = 2000
# take a look at the most used clues
most_clue = df[['cleaned_text','frequency','Lens']][df['frequency'] >= 5]
most_clue

,cleaned_text,frequency,Lens
48,amazon is disrupt the healthcar industri amazon care amazon is disrupt the healthcar industri in multipl way as an onlin pharmaci by sell medic product and as a digit enabl of of healthcar amazon launch a virtual primari care clinic call amazon care the latest sign of the giant retail s grow ambit in healthcar amazon care includ video and text visit with clinician as well as in home care the program is describ as a new benefit for employe of the compani and is a pilot project in the seattl area amazon ha been expand in healthcar and separ partner with jpmorgan and berkshir hathaway on an effort to improv care for their worker call haven in june amazon acquir the onlin pharmaci pillpack for million with it disrupt network and smart technolog they could be a major disruptor,5,Social
81,eu lawmak call for target to reduc raw materi use lawmak in the european parliament s environ committe will call for legal bind target to reduc raw materi use in a report due to be vote on thi week the vote on tuesday januari will shape the parliament s respons to the european commiss s circular economi action plan present in march last year tighter wast and recycl law are central to the commiss s plan with new target due to be present thi year as part of a broader revis of eu wast rule in their report mep call on the european commiss to propos scienc base bind eu mid term and long term target for the reduct in the use of primari raw materi in manufactur industri to achiev thi they call for a back cast approach to ensur that polici object are on a credibl path to achiev a carbon neutral and fulli circular economi within planetari boundari by at the latest hollist is current in teh medic consum busi and could find itself in jeopardi if or when thi legisl will come into play,8,Political
93,amazon offer to assist biden administr with covid vaccin distribut and logist may give amazon a privat compani a valuabl new trove of health data as it is expand into pharmaci and digit health potenti suppli chain disrupt in futur amazon s bid come as biden offici say they re tri to repair the vaccin distribut system they inherit could help the compani boost it own ambit of expand into the trillion health care marketplac amazon might use ani feder agreement to push it own corpor advantag becaus they re veri well known as a compani to tri to liter domin the market and forc out ani competit,5,Political
122,in the next year we ll reinvent everi industri in the next decad we ll experi more progress than in the past year whi comput is the foundat be it classic or quantum comput as it becom faster and cheaper lot of technolog that use it also becom more capabl for exampl commun network sensor robot augment and virtual realiti blockchain and ai are all exponenti improv but they are also be interweav and converg for exampl ai with robot and that s get even faster becaus the number of peopl with access to technolog is increas so we are abl to solv more problem there s also more capit avail than ever befor which mean more crazi idea be fund which in turn lead to more breakthrough and the cost of do thing is also get cheaper the number of experi go on in the silicon valley garag is explod,6,Economical
123,musk launch hunt for carbon neg not neutral technolog with m xprize vc s capit solv the climat crisi could they do the same to help startup with the typic barrier to entri into medic devic develop,5,Economical
139,algorithm will run machin not human underperform sensor commun and algorithm unintent cyber safeti or deliber manipul cyber secur are the princip sourc of the new risk in addit there is the increas complex of softwar system multipli in uncertainti by machin learn and ai base system,5,Technological
157,antimicrobi packag there are a few essenti oil that can actual possess antimicrobi properti one of these essenti oil is orang peel essenti oil it is use to produc antimicrobi film and coat orang juic process wast as a powder can be an 

In [13]:
df['Lens'].value_counts()

Technological    128
Social            42
Environmental     29
Economical        18
Political          9
Legal              5
Societal           1
Name: Lens, dtype: int64

### train test split

In [14]:
df['multi_frequency'] = df['frequency'].copy()
df['binary_frequency'] = np.where(df['frequency'] > 0, 1, 0)

In [15]:
df[['binary_frequency', 'multi_frequency']]

,binary_frequency,multi_frequency
0,0,0
1,0,0
2,0,0
3,1,2
4,1,1
...,...,...
227,1,1
228,1,1
229,1,1
230,0,0


In [16]:
X_train_bi, X_test_bi, y_train_bi, y_test_bi = train_test_split(df['cleaned_text'], df['binary_frequency'], test_size=0.3, random_state=42, stratify=df['binary_frequency'])
X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(df['cleaned_text'], df['multi_frequency'], test_size=0.3, random_state=42)

In [17]:
# manually create 3-fold cross validation data
X_1, X_2, X_3 = np.split(df['cleaned_text'].sample(frac=1, random_state=42), [int(.33*len(df['binary_frequency'])), int(.66*len(df['binary_frequency']))])
Y_1, Y_2, Y_3 = np.split(df['binary_frequency'].sample(frac=1, random_state=42), [int(.33*len(df['binary_frequency'])), int(.66*len(df['binary_frequency']))])
X_train1 = pd.concat([X_1, X_2])
Y_train1 = pd.concat([Y_1,Y_2])
X_test1 = X_3 
Y_test1 = Y_3 
X_train2 = pd.concat([X_1,X_3])
Y_train2 = pd.concat([Y_1,Y_3])
X_test2 = X_2 
Y_test2 = Y_2 
X_train3 = pd.concat([X_2,X_3])
Y_train3 = pd.concat([Y_2,Y_3])
X_test3 = X_1
Y_test3 = Y_1

### text classification

Here we try out three methods for the bi-frequency output:    
1. Bag of words
2. Word2vec
3. BERT/XLNet/RoBERTa

1. Bag of words

In [18]:
def bow_model(x_train, x_test, y_train, y_test, n1, n2, analyzer = 'word', norm = 'l2',cutoff = 0.5, use_idf=True, model = 'MultinomialNB', multi_class = False):
  count_vec = CountVectorizer(ngram_range=(n1, n2), stop_words='english', analyzer= analyzer)
  bow_count = count_vec.fit_transform(x_train)
  tf_transformer = TfidfTransformer(norm = norm, use_idf = use_idf).fit(bow_count)
  bow = tf_transformer.transform(bow_count)
  
  if model == 'MultinomialNB' and multi_class == False:
    clf = MultinomialNB().fit(bow, y_train)
    X_new_counts = count_vec.transform(x_test)
    X_new_tfidf = tf_transformer.transform(X_new_counts)
    confidence = clf.predict_proba(X_new_tfidf)
    predicted = [item[0] for item in np.where(confidence>cutoff,0,1)]
    #return y_test, predicted, confidence

  if model == 'LogisticRegression' and multi_class == False:
    clf = LogisticRegression().fit(bow, y_train)
    X_new_counts = count_vec.transform(x_test)
    X_new_tfidf = tf_transformer.transform(X_new_counts)
    confidence = clf.predict_proba(X_new_tfidf)
    predicted = [item[0] for item in np.where(confidence>cutoff,0,1)]
   # return y_test, predicted, confidence

  if model == 'SGDClassifier' and multi_class == False:
    clf = SGDClassifier(loss='modified_huber', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None).fit(bow, y_train)
    X_new_counts = count_vec.transform(x_test)
    X_new_tfidf = tf_transformer.transform(X_new_counts)
    confidence = clf.predict_proba(X_new_tfidf)
    predicted = clf.predict(X_new_tfidf)
  #  return y_test, predicted, confidence

  micro_precision, micro_recall, micro_f1, support1 = precision_recall_fscore_support(y_test, predicted, average='micro')
  macro_precision, macro_recall, macro_f1, support2 = precision_recall_fscore_support(y_test, predicted, average='macro')

  return micro_precision, macro_precision, micro_recall, macro_recall, micro_f1, macro_f1, predicted, confidence

In [19]:
bow_summary_table = pd.DataFrame(index=['model', 'norm', 'analyzer', 'ngram1', 'ngram2', 'micro_precision', 'macro_precision', 'micro_recall', 'macro_recall', 'micro_f1', 'macro_f1'])
i = 0
#prediction example
for n2 in (1,2):
  for analyzer in ['word', 'char']:
    for norm in ['l1', 'l2']:
      for model in ['MultinomialNB', 'LogisticRegression', 'SGDClassifier']:
        i = i + 1
        model_name = 'model' + str(i)
        micro_precision1, macro_precision1, micro_recall1, macro_recall1, micro_f11, macro_f11, predicted, confidence = bow_model(X_train1, X_test1, Y_train1, Y_test1, model = model, n1 = 1, n2 = n2, analyzer = analyzer, norm = norm, cutoff = 0.33)
        micro_precision2, macro_precision2, micro_recall2, macro_recall2, micro_f12, macro_f12, predicted, confidence = bow_model(X_train2, X_test2, Y_train2, Y_test2, model = model, n1 = 1, n2 = n2, analyzer = analyzer, norm = norm, cutoff = 0.33)
        micro_precision3, macro_precision3, micro_recall3, macro_recall3, micro_f13, macro_f13, predicted, confidence = bow_model(X_train1, X_test1, Y_train1, Y_test1, model = model, n1 = 1, n2 = n2, analyzer = analyzer, norm = norm, cutoff = 0.33)
       
        bow_summary_table[model_name] = [model, norm, analyzer, '1', n2,(micro_precision1+micro_precision2+micro_precision3)/3, (macro_precision1+macro_precision2+macro_precision3)/3, (micro_recall1+micro_recall2+micro_recall3)/3 , (macro_recall1+macro_recall2+macro_recall3)/3, (micro_f11+micro_f12+micro_f13)/3 , (macro_f11+macro_f12+macro_f13)/3]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

In [20]:
bow_summary_table

,model1,model2,model3,model4,model5,model6,model7,model8,model9,model10,model11,model12,model13,model14,model15,model16,model17,model18,model19,model20,model21,model22,model23,model24
model,MultinomialNB,LogisticRegression,SGDClassifier,MultinomialNB,LogisticRegression,SGDClassifier,MultinomialNB,LogisticRegression,SGDClassifier,MultinomialNB,LogisticRegression,SGDClassifier,MultinomialNB,LogisticRegression,SGDClassifier,MultinomialNB,LogisticRegression,SGDClassifier,MultinomialNB,LogisticRegression,SGDClassifier,MultinomialNB,LogisticRegression,SGDClassifier
norm,l1,l1,l1,l2,l2,l2,l1,l1,l1,l2,l2,l2,l1,l1,l1,l2,l2,l2,l1,l1,l1,l2,l2,l2
analyzer,word,word,word,word,word,word,char,char,char,char,char,char,word,word,word,word,word,word,char,char,char,char,char,char
ngram1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
ngram2,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2
micro_precision,0.306428,0.306428,0.693572,0.672804,0.501836,0.664146,0.306428,0.306428,0.568031,0.306428,0.331744,0.70634,0.306428,0.306428,0.693572,0.655488,0.497726,0.638172,0.685133,0.306428,0.57236,0.693572,0.314867,0.693572
macro_precision,0.153214,0.153214,0.431839,0.558228,0.609399,0.573022,0.153214,0.153214,0.284016,0.153214,0.573041,0.851271,0.153214,0.153214,0.346786,0.495788,0.60716,0.498968,0.345488,0.153214,0.39498,0.346786,0.457544,0.438352
micro_recall,0.306428,0.306428,0.693572,0.672804,0.501836,0.664146,0.306428,0.306428,0.568031,0.306428,0.331744,0.70634,0.306428,0.306428,0.693572,0.655488,0.497726,0.638172,0.685133,0.306428,0.57236,0.693572,0.314867,0.693572
macro_recall,0.5,0.5,0.5076,0.542874,0.601853,0.567441,0.5,0.5,0.5,0.5,0.514382,0.520833,0.5,0.5,0.5,0.495869,0.598937,0.509889,0.493939,0.5,0.499345,0.5,0.494433,0.530398
micro_f1,0.306428,0.306428,0.693572,0.672804,0.501836,0.664146,0.306428,0.306428,0.568031,0.306428,0.331744,0.70634,0.306428,0.306428,0.693572,0.655488,0.497726,0.638172,0.685133,0.306428,0.57236,0.693572,0.314867,0.693572


2. word embedding

In [ ]:
# all below are test codes

In [22]:
def w2v_model(data, min_count, size, negative, iter, window, sg):
  df_tokenize = []
  for text in data:
    tokenized = word_tokenize(text)
    df_tokenize.append(tokenized)
  
  model = gensim.models.Word2Vec(sentences = df_tokenize, min_count = min_count, size = size, negative = negative, iter = iter, window = window, sg = sg)
  model.wv.save_word2vec_format('trial.txt', binary=False)
  return model

# word2vec trial1
w2v1 = w2v_model(df['cleaned_text'], 1, 100, 5, 5, 5, 0)

kk = w2v1.wv.vocab.keys()
kk = set(kk)

In [23]:
def get_weight_matrix(embedding, vocab):
	# total vocabulary size plus 0 for unknown words
	vocab_size = len(vocab) + 1
	# define weight matrix dimensions with all 0
	weight_matrix = zeros((vocab_size, 100))
	# step vocab, store vectors using the Tokenizer's integer mapping
	for word, i in vocab.items():
		weight_matrix[i] = embedding.get(word)
	return weight_matrix

In [30]:
# load embedding as a dict
def load_embedding(filename):
	# load embedding into memory, skip first line
	file = open(filename,'r')
	lines = file.readlines()[1:]
	file.close()
	# create a map of words to vectors
	embedding = dict()
	for line in lines:
		parts = line.split()
		# key is string word, value is numpy array for vector
		embedding[parts[0]] = asarray(parts[1:], dtype='float32')
	return embedding

In [ ]:
#######debug

In [ ]:
#######debug

In [24]:
train_docs = X_train_bi.tolist()

In [25]:
# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(train_docs)
 
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(train_docs)
# pad sequences
max_length = max([len(s.split()) for s in train_docs])

In [26]:
Xtrain_array = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# define training labels
ytrain_array = np.array(y_train_bi.to_list())

In [27]:
test_docs = X_test_bi.to_list()
encoded_docs2 = tokenizer.texts_to_sequences(test_docs)
Xtest_array = pad_sequences(encoded_docs2, maxlen=max_length, padding='post')
Ytest_array = np.array(y_test_bi.to_list())

In [28]:
# define vocabulary size (largest integer value)
vocab_size = len(tokenizer.word_index) + 1
vocab_size

2933

In [31]:
raw_embedding = load_embedding('trial.txt')

In [32]:

# get vectors in the right order
embedding_vectors = get_weight_matrix(raw_embedding, tokenizer.word_index)
# create the embedding layer
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable=False)

In [33]:
opt = keras.optimizers.Adam(learning_rate=0.0001)

# define model
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
print(model.summary())
# compile network
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), optimizer= opt, metrics=['accuracy'])
# fit network
model.fit(Xtrain_array, ytrain_array, epochs=20, batch_size=64, verbose=2)
# evaluate
loss, acc = model.evaluate(Xtest_array, Ytest_array, verbose=0)
preds = model.predict(Xtest_array)
print('Test Accuracy: %f' % (acc*100))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 386, 100)          293300    
                                                                 
 conv1d (Conv1D)             (None, 382, 32)           16032     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 191, 32)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 6112)              0         
                                                                 
 dense (Dense)               (None, 1)                 6113      
                                                                 
Total params: 315,445
Trainable params: 22,145
Non-trainable params: 293,300
_____________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


3/3 - 1s - loss: 0.6821 - accuracy: 0.5370 - 1s/epoch - 418ms/step
Epoch 2/20
3/3 - 0s - loss: 0.6688 - accuracy: 0.6296 - 187ms/epoch - 62ms/step
Epoch 3/20
3/3 - 0s - loss: 0.6575 - accuracy: 0.6481 - 191ms/epoch - 64ms/step
Epoch 4/20
3/3 - 0s - loss: 0.6487 - accuracy: 0.6605 - 184ms/epoch - 61ms/step
Epoch 5/20
3/3 - 0s - loss: 0.6390 - accuracy: 0.6605 - 192ms/epoch - 64ms/step
Epoch 6/20
3/3 - 0s - loss: 0.6327 - accuracy: 0.6605 - 192ms/epoch - 64ms/step
Epoch 7/20
3/3 - 0s - loss: 0.6282 - accuracy: 0.6605 - 183ms/epoch - 61ms/step
Epoch 8/20
3/3 - 0s - loss: 0.6227 - accuracy: 0.6605 - 183ms/epoch - 61ms/step
Epoch 9/20
3/3 - 0s - loss: 0.6185 - accuracy: 0.6605 - 184ms/epoch - 61ms/step
Epoch 10/20
3/3 - 0s - loss: 0.6154 - accuracy: 0.6605 - 190ms/epoch - 63ms/step
Epoch 11/20
3/3 - 0s - loss: 0.6120 - accuracy: 0.6605 - 192ms/epoch - 64ms/step
Epoch 12/20
3/3 - 0s - loss: 0.6086 - accuracy: 0.6605 - 185ms/epoch - 62ms/step
Epoch 13/20
3/3 - 0s - loss: 0.6057 - accuracy: 0.